### Task 1. Set up your project
  + Start by getting the lab files from GitHub:
   ```
    cd
    SRC_REPO=https://github.com/GoogleCloudPlatform/mlops-on-gcp
    kpt pkg get $SRC_REPO/workshops/mlep-qwiklabs/tfserving-canary-gke tfserving-canary
   ```

In [ ]:
# Configure environment variables
export PROJECT_ID=$(gcloud config get-value project)
export PROJECT_NUMBER=$(gcloud projects describe ${PROJECT_ID} \
    --format="value(projectNumber)")
export CLUSTER_NAME=cluster-1
export CLUSTER_ZONE=us-west1-c
export WORKLOAD_POOL=${PROJECT_ID}.svc.id.goog
export MESH_ID="proj-${PROJECT_NUMBER}"

In [ ]:
# Task 2. Set up your GKE cluster
# Now run the following command in Cloud Shell to create the Kubernetes cluster cluster-1:
gcloud config set compute/zone ${CLUSTER_ZONE}
gcloud beta container clusters create ${CLUSTER_NAME} \
    --machine-type=n1-standard-4 \
    --num-nodes=6 \
    --workload-pool=${WORKLOAD_POOL} \
    --logging=SYSTEM,WORKLOAD \
    --monitoring=SYSTEM \
    --subnetwork=default \
    --release-channel=stable \
    --labels mesh_id=${MESH_ID}
# After your cluster is created, run this command to ensure you have the cluster-admin role on your cluster:
kubectl create clusterrolebinding cluster-admin-binding   --clusterrole=cluster-admin   --user=$(whoami)@qwiklabs.net

In [ ]:
# Prepare to install Anthos Service Mesh
curl https://storage.googleapis.com/csm-artifacts/asm/asmcli_1.15 > asmcli
# Make the script executable:
chmod +x asmcli
# Install Anthos Service Mesh
./asmcli install \
  --project_id $PROJECT_ID \
  --cluster_name $CLUSTER_NAME \
  --cluster_location $CLUSTER_ZONE \
  --fleet_id $PROJECT_ID \
  --output_dir ./asm_output \
  --enable_all \
  --option legacy-default-ingressgateway \
  --ca mesh_ca \
  --enable_gcp_components
# Task 3. Install an ingress gateway
# #1. Create a namespace for the ingress gateway if you don't already have one. 
# Gateways are user workloads, and as a best practice, they shouldn't be deployed
#  in the control plane namespace.
GATEWAY_NS=istio-gateway
kubectl create namespace $GATEWAY_NS